In [76]:
import datetime as dt
import json as js
import urllib.request

url='https://api.openweathermap.org/data/2.5/onecall?lat=64.286670&lon=27.622125&units=metric&appid=e25afeccd6e22a2e996bed2809e43452'


resp=urllib.request.urlopen(url)
jsfile_Tornio=js.load(resp)

def unix_to_UTC_time(time_survey):
    return dt.datetime.utcfromtimestamp(time_survey).strftime('%Y-%m-%d %H:%M:%S')

date_weather_info=[]
for each_day in jsfile_Tornio['daily']:
    date_predicted=unix_to_UTC_time(each_day['dt'])
    temp_max=each_day['temp']['max']
    temp_min=each_day['temp']['min']
    temp_avg=each_day['temp']['day']
    rain_info=each_day.get('rain',0) # if it was no rain on the surveyed day, then return value 0

    date_weather_info.append({
        'date_predicted':date_predicted,
        'temp_max':temp_max,
        'temp_min':temp_min,
        'temp_avg':temp_avg,
        'rain_info (mm)': rain_info
    })

data_weather=[]
k=1
today=dt.datetime.now().strftime('%Y-%m-%d')
for i in date_weather_info:
    data_tuple=(k,today,i['date_predicted'],i['temp_min'],i['temp_max'],i['temp_avg'],i['rain_info (mm)'])
    data_weather.append(data_tuple)
    k+=1
data_weather

[(1, '2020-04-09', '2020-04-09 10:00:00', -4.71, 2.55, 2.55, 0),
 (2, '2020-04-09', '2020-04-10 10:00:00', -6.34, -0.31, -1.73, 0),
 (3, '2020-04-09', '2020-04-11 10:00:00', -4.65, 0.71, -0.29, 0),
 (4, '2020-04-09', '2020-04-12 10:00:00', -3.66, 0.18, -0.9, 0),
 (5, '2020-04-09', '2020-04-13 10:00:00', -6.01, -2.97, -3.63, 0),
 (6, '2020-04-09', '2020-04-14 10:00:00', -8.44, -1.32, -3.93, 0),
 (7, '2020-04-09', '2020-04-15 10:00:00', -3.24, 3.87, 0.36, 0),
 (8, '2020-04-09', '2020-04-16 10:00:00', 1.94, 4.83, 4.48, 0)]

In [2]:
import sqlite3 as sql
Wea_pred=sql.connect('WeatherDataTest.db')
try:   
    Wea_pred.execute('''
    CREATE TABLE "WeatherPrediction" (
        "prediction_id"	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
        "predictionDate"	DATE NOT NULL,
        "date"	DATE NOT NULL,
        "minTemp"	INTEGER NOT NULL DEFAULT 0,
        "maxTemp"	INTEGER NOT NULL DEFAULT 0,
        "avgTemp"	INTEGER NOT NULL DEFAULT 0,
        "rain"	INTEGER NOT NULL DEFAULT 0
    );
    ''')

    Wea_pred.executemany('''insert into WeatherPrediction (prediction_id, predictionDate, date, minTemp, maxTemp, avgTemp, rain
    values (?,?, ?, ?, ?, ?, ?)''',data_weather)

except:
    pass
Wea_pred.commit()
Wea_pred.close()

In [3]:
%%capture
%load_ext sql
%sql sqlite:///WeatherDataTest.db

In [4]:
%%sql
select * from WeatherPrediction

* sqlite:///WeatherDataTest.db
Done.


prediction_id,predictionDate,date,minTemp,maxTemp,avgTemp,rain
1,2020-04-09,2020-04-09 10:00:00,-4.05,2.62,2.62,0
2,2020-04-09,2020-04-10 10:00:00,-5.62,-0.78,-2.32,0
3,2020-04-09,2020-04-11 10:00:00,-4.41,0.64,-0.43,0
4,2020-04-09,2020-04-12 10:00:00,-4.06,0.29,-0.98,0
5,2020-04-09,2020-04-13 10:00:00,-4.73,-0.92,-0.92,0
6,2020-04-09,2020-04-14 10:00:00,-7.14,-1.27,-3.53,0
7,2020-04-09,2020-04-15 10:00:00,-1.86,2.73,2.21,0.92
8,2020-04-09,2020-04-16 10:00:00,-3.35,4.45,2.73,0.3
